In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

titles = []
links = []
regulars = []
sales = []
reviews = []
descriptions = []

# Loop through pages 1 to 3
for page in range(1, 4):
    url = f'https://kidspark.co.in/collections/all?page={page}'
    print(f"Scraping page {page}: {url}")
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    cards = soup.find_all('div', class_='card-wrapper')

    for card in cards:
        title_tag = card.find('h3', class_='card__heading h5')
        link_tag = card.find('a')

        title = title_tag.get_text(strip=True) if title_tag else 'No title'
        href = link_tag.get('href') if link_tag else None
        full_link = f"https://kidspark.co.in{href}" if href else 'No link'

        regular = card.find('div', class_='price__regular')
        regulars.append(regular.get_text(strip=True) if regular else 'No info')

        sale = card.find('s', class_='price-item price-item--regular')
        sales.append(sale.get_text(strip=True) if sale else 'No info')

        titles.append(title)
        links.append(full_link)

        # Visit individual product page
        if full_link != 'No link':
            try:
                product_response = requests.get(full_link, headers=headers)
                product_soup = BeautifulSoup(product_response.text, 'lxml')

                # Description
                desc_tag = product_soup.find('div', class_='product__description')
                desc_text = desc_tag.get_text(strip=True) if desc_tag else 'No info'
                descriptions.append(desc_text)

                # Review
                review_tag = product_soup.find('div', class_='jdgm-rev-widg__summary')
                review_text = review_tag.get_text(strip=True) if review_tag else 'No info'
                reviews.append(review_text)

            except Exception as e:
                descriptions.append('Error')
                reviews.append('Error')

        time.sleep(1)  # Be polite to the server

# Save all data to DataFrame
df = pd.DataFrame({
    'title': titles,
    'links': links,
    'sale': regulars,
    'regular': sales,
    'description': descriptions,
    'review': reviews
})

# Save to Excel
df.to_excel('ecommercewebscrapping.xlsx', index=False)
print('Scraping completed and saved to ecommercewebscrapping.xlsx')


Scraping page 1: https://kidspark.co.in/collections/all?page=1
Scraping page 2: https://kidspark.co.in/collections/all?page=2
Scraping page 3: https://kidspark.co.in/collections/all?page=3
Scraping completed and saved to ecommercewebscrapping.xlsx
